## Check GPU Use:

In [ ]:
import tensorflow as tf
print(bool(len(tf.config.list_physical_devices('GPU'))))

## Import Libaries:

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# Basic libaries:
import os
import copy
import random

import pandas as pd
import numpy as np

# Plotting and Image Display / Manipulation libaries:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow

from skimage.segmentation import mark_boundaries
from skimage import morphology, io, color, exposure, img_as_float, transform

# Image processing liabries:
from PIL import Image, ImageOps, ImageFilter

# General machine learning libaries:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, f1_score

# Neural Network libaries:
import keras
from keras import backend 
from tensorflow.keras.models import load_model

from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, Flatten, Dense

from keras.optimizers import SGD

from keras.applications.vgg16 import VGG16 as VGG
from keras.applications.vgg16 import preprocess_input as VGG_preprocess_input

from keras.applications import DenseNet121 as DenseNet
from keras.applications.densenet import preprocess_input as DenseNet_preprocess_input

from keras.applications import ResNet50V2 as ResNet
from keras.applications.resnet import preprocess_input as ResNet_preprocess_input

# Explanation libaries:
import lime
from lime import lime_image


## Prepare Images:

In [ ]:
# Define data directories:
data_dir_path = "../../group/hedge/Data"

# Define directory to save models into:
model_dir_path = "../results/models"

# Define path to lung segmentation model file:
lung_seg_model_path = "../results/models/trained_model.hdf5"


In [ ]:
# Load all images from a data directory:
def load_all_images_from_dir(data_dir, target_size):
    
    image_list = []
    for image in os.listdir(data_dir):
        
        file_typ = image.split(".")[-1]
        
        if (file_typ == "jpeg") or (file_typ == "jpg") or (file_typ == "png"):    
            image = load_img(os.path.join(data_dir, image), target_size=target_size)
            image_list.append(image)
        
    return image_list


In [ ]:
# Combine and transform diffrent image-classes to np-arrays and get class-labels: 
def get_dp_label_pair(class_image_lists, nr_samples = None):

    nr_classes = len(class_image_lists)
    
    # Get "nr_samples" images from each class:
    for i in range(0, nr_classes):
        
        if nr_samples is not None:
            class_image_lists[i] = class_image_lists[i][:nr_samples]
        
    # Get datapoints as np-arrays: 
    datapoints = []
    for j in range(0, nr_classes):    
        for current_dp in class_image_lists[j]:
            
            # Convert current image into a np-arry:
            current_dp = img_to_array(current_dp)
            current_dp = np.expand_dims(current_dp, axis=0)    
            
            datapoints.append(current_dp)
            
    datapoints = np.vstack(datapoints)
    
    # Get the labels from datapoints:
    label_list = []
    for k in range(0, nr_classes):
        
        # Set labels for current class to 1 if first class, else set to zero:
        if k == 0:
            class_labels = np.ones(len(class_image_lists[k]))
        else: 
            class_labels = np.zeros(len(class_image_lists[k]))
            
        for l in range(1, nr_classes):
            
            if l == k:
                class_labels = np.vstack((class_labels, np.ones(len(class_image_lists[l]))))
            else:
                class_labels = np.vstack((class_labels, np.zeros(len(class_image_lists[l]))))
        
        label_list.append(class_labels)
    
    # Reshape label arrays:
    labels = label_list[0].T
    
    for ls in label_list[1:]:
        labels = np.append(labels, ls.T, axis=0)
    
    return datapoints, labels    


In [ ]:
def preprocess_images(images, resize_scale=None, blur_radius=None, deep_copy=True):
    
    if deep_copy:
        preprocessed_images = []
    else:
        preprocessed_images = images
    
    if type(images) is not list:
        images = [images]
        
    for z, image in enumerate(images):

        if deep_copy:
            image = image.copy()
        
        # Resize images, if specified:
        if resize_scale is not None:
            image = image.resize(resize_scale)
        
        # Convert images to greyscale:
        image = image.convert('L').convert('RGB')
        
        # Equalize the image histogram:
        image = ImageOps.equalize(image)
        
        # Use blur, if specified:
        if blur_radius is not None:
            image = image.filter(ImageFilter.GaussianBlur(radius=blur_radius)) # default radius = 2;
        
        if deep_copy:
            preprocessed_images.append(image)
        else:
            preprocessed_images[z] = image
        
    return preprocessed_images
             

In [ ]:
def segment_lung_area(images, model_path = lung_seg_model_path):
    
    lung_masks = []
    
    # Preprocess test data:  
    X = []
    for image in images:
        
        temp_image = img_as_float(image)[...,0]
        temp_image = transform.resize(temp_image, (256, 256))
        temp_image = exposure.equalize_hist(temp_image)
        temp_image = np.expand_dims(temp_image, -1)
        
        X.append(temp_image)
        
    X = np.array(X)    
    X -= X.mean()
    X /= X.std()

    # Load model
    UNet = load_model(model_path)
    
    i = 0
    for xx in ImageDataGenerator(rescale=1.).flow(X, batch_size=1):
        
        pred_lung_area = UNet.predict(xx)[..., 0].reshape(X[0].shape[:2])
        
        size = 0.02 * np.prod(xx.shape[1:3])
        
        pred_lung_area = pred_lung_area > 0.5
        pred_lung_area = morphology.remove_small_objects(pred_lung_area, size)
        pred_lung_area = morphology.remove_small_holes(pred_lung_area, size)

        # Filter pixels in original image with mask: 

        # [rows, cols] = pr.shape
        # color_mask = np.zeros((rows, cols, 3))
        # color_mask[pr == 1] = [1, 1, 1]
        
        lung_masks.append(pred_lung_area)
        
        i += 1
        if i == X.shape[0]:
            break
            
    return lung_masks
    

## Get and Retrain Model:

In [ ]:
def redefine_model_wo_top(model, input_shape = (224,224,3), nr_classes = 2, fc_layer_structure = [2048, 2048]):

    # Define the new input format for the given model (e.g. 3 channels - 224 width x 224 height):
    model_input = Input(shape=input_shape, name = 'model_input')
    
    # Reuse convolutional layer of the given model: 
    model_output = model(model_input)

    # Define new fully connected layer structure:
    x = Flatten(name='flatten')(model_output)
    
    for i, fc_layer_neurons in enumerate(fc_layer_structure): 
        x = Dense(fc_layer_neurons, activation='relu', name='fc' + str(i))(x)
    
    x = Dense(nr_classes, activation='softmax', name='predictions')(x)

    #Create your own model 
    new_model = Model(input = model_input, output = x)

    return new_model


In [ ]:
# Define custom weighted categorical crossentropy loss:
def weighted_categorical_crossentropy(w):
    w = backend.variable(w)
        
    def loss(y, y_pred):
        # Preprocess predictions to enable use in loss:
        y_pred /= backend.sum(y_pred, axis=-1, keepdims=True)
        y_pred = backend.clip(y_pred, backend.epsilon(), 1 - backend.epsilon())
        
        # Calculate and return calculated loss:
        loss = -backend.sum(y * backend.log(y_pred) * w, -1)
        return loss
    
    return loss

# Train model for binary classification (loss = categorical_crossentropy):
def train_model(model, x_train, y_train, x_test = None, y_test = None, weighted_loss=None, batch_size = 1, epochs = 10, shuffle = True, optimizer = SGD(lr = 0.0001, decay = 1e-6, momentum = 0.01, nesterov = True)):
    
    if weighted_loss is None:
        model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
    else:
        model.compile(loss=weighted_categorical_crossentropy(weighted_loss), optimizer = optimizer, metrics=['accuracy'])
    
    score = []
    if x_test is not None:
        history = model.fit(x_train, y_train, validation_data=(x_test,y_test), batch_size = batch_size, epochs = epochs, shuffle = shuffle)
        score = model.evaluate(x_test, y_test, batch_size = batch_size)    
        
    else:
        history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs, shuffle = shuffle)
        
        
    return model, score


In [ ]:
def evaluate_model(model, x_test, y_test, label_list = ["Normal", "Pneumonia"]):
    
    y_true = [np.argmax(pred) for pred in list(y_test)]
    
    y_pred_prob = vgg_model.predict(x_test)
    y_pred = [np.argmax(pred) for pred in list(y_pred_prob)]
    
    classification_report_results = classification_report(y_true, y_pred, target_names=label_list)
    confusion_matrix_results = confusion_matrix(y_true, y_pred)    
    
    roc_auc_score_result = roc_auc_score(y_true, y_pred_prob, multi_class="ovr")
    
    weighted_f1_score_results = f1_score(y_true, y_pred, average='weighted')
    
    return weighted_f1_score_results, roc_auc_score_result, classification_report_results, confusion_matrix_results


## Predict Data:

In [ ]:
def prepare_images_for_model(loaded_images, preprocess_input_function = None):
    
    images = []
    for image in loaded_images:
    
        # Convert the image into a np-array:
        image = img_to_array(image)

        # Reshape the image array (add first dimension for the batchsize):
        image = np.expand_dims(image, axis=0) 

        # Preprocess the image for the VGG16-model:
        if preprocess_input_function is None:
            pass
        else:
            image = preprocess_input_function(image)
        
        images.append(image)
    
    images = np.vstack(images)
    return images



In [ ]:
def predict_xray_image(model, image, label_list = ["Normal", "Pneumonia"], display_results = False, return_prob = False):
    
    prepared_image = prepare_images_for_model([image], preprocess_input_function = None)

    # Predict class-propabilities for loaded images using the new model:
    new_model_predictions_prob = list(model.predict(prepared_image)[0])
    predict_model_label = label_list[new_model_predictions_prob.index(max(new_model_predictions_prob))]
    
    if display_results:
        
        # Display loaded single image:
        imgplot_xray = plt.imshow(image)
        plt.show()

        # Display top class propabilities from single loaded image: 
        print("Label:", predict_model_label, "\n")        
    
    if return_prob:
        return predict_model_label, new_model_predictions_prob
    else:
        return predict_model_label
    
    

## Use LIME to explain predictions:

In [ ]:
def explain_predictions(model, explainer, x_dataset, nr_samples = None, indecies = None, classes = ["Normal", "Pneumonia"], num_samples_lime = 1000, num_features_lime = 10, figwidth = 10, figheight = 10):

    if (nr_samples is None) and (indecies is None):
        return -1
    
    # Get number of classes:
    nr_classes = len(classes)
    
    # If no indecies are given, select "nr_samples" samples:
    if indecies is None:
        indecies = random.sample(range(x_data.shape[0]), nr_samples)
    else:
        nr_samples = len(indecies)
    
    # Set plot height and width:
    fig, ax = plt.subplots(nr_samples, nr_classes + 1, sharex='col', sharey='row')
    fig.set_figwidth(figwidth)
    fig.set_figheight(figheight)
                
    xs = [np.array(prepare_images_for_model([x]), dtype=np.uint8)[0] for x in x_dataset]

    # Get explanations for all selected images:
    for j in range(nr_samples):
        explanation = explainer.explain_instance(xs[indecies[j]], model.predict, 
                                                 top_labels=nr_classes, hide_color=0, num_samples=num_samples_lime)

        
        ax[j,0].imshow(xs[indecies[j]])
        ax[j,0].set_title(classes[list(y_data[indecies[j]]).index(max(list(y_data[indecies[j]])))])

        xs_pred_label, xs_pred_prob = predict_xray_image(model, xs[indecies[j]], label_list = classes, 
                                                         display_results = False, return_prob = True)
        
        for i in range(nr_classes):
            temp, mask = explanation.get_image_and_mask(i, positive_only=False, 
                                                        num_features=num_features_lime, hide_rest=False)

            temp = np.array(temp, dtype=np.uint8)

            ax[j,i+1].imshow(mark_boundaries(temp, mask))
            ax[j,i+1].set_title('p({}) = {:.4f}'.format(classes[i], xs_pred_prob[i]))
            
            
    return fig, ax
    

## Save and Load Models:

In [ ]:
def save_model(model, model_name):
    model.save(model_dir_path + '/' + model_name)  # creates a HDF5 file 'my_model.h5'

def load_model(model_name):
    # Returns a compiled model:
    model = load_model(model_dir_path + '/' + model_name)
    return model


# Pipeline:

Load the images and split them into training and test data sets:

In [ ]:
# Get list of image from "normal chest xray" data directory:
normal_data_dir = os.path.join(data_dir_path, "Normal_old")
normal_image_list_full = load_all_images_from_dir(normal_data_dir, (256, 256))

# Get list of image from "Bacterial pneumonia chest xray" data directory:
bacterial_pneumonia_data_dir = os.path.join(data_dir_path, "Bacterial_Pneumonia_old")
bacterial_pneumonia_image_list_full = load_all_images_from_dir(bacterial_pneumonia_data_dir, (256, 256))

# Get list of image from "Viral pneumonia chest xray" data directory:
viral_pneumonia_data_dir = os.path.join(data_dir_path, "Viral_Pneumonia_old")
viral_pneumonia_image_list_full = load_all_images_from_dir(viral_pneumonia_data_dir, (256, 256))

# Get list of image from "COVID19 chest xray" data directory:
covid19_data_dir = os.path.join(data_dir_path, "COVID_19_old")
covid19_image_list_full = load_all_images_from_dir(covid19_data_dir, (256, 256))


In [ ]:
# Shuffle the lists of diffrent class images:
random.shuffle(normal_image_list_full)
random.shuffle(bacterial_pneumonia_image_list_full)
random.shuffle(viral_pneumonia_image_list_full)
random.shuffle(covid19_image_list_full)

# Create train-test-split:
test_split = 1/3

nr_normal_images = len(normal_image_list_full)
nr_bacterial_images = len(bacterial_pneumonia_image_list_full)
nr_viral_images = len(viral_pneumonia_image_list_full)
nr_covid19_images = len(covid19_image_list_full)

normal_image_list_full = [normal_image_list_full[:-round(nr_normal_images * test_split)], normal_image_list_full[-round(nr_normal_images * test_split):]]
bacterial_pneumonia_image_list_full = [bacterial_pneumonia_image_list_full[:-round(nr_bacterial_images * test_split)], bacterial_pneumonia_image_list_full[-round(nr_bacterial_images * test_split):]]
viral_pneumonia_image_list_full = [viral_pneumonia_image_list_full[:-round(nr_viral_images * test_split)], viral_pneumonia_image_list_full[-round(nr_viral_images * test_split):]]
covid19_image_list_full = [covid19_image_list_full[:-round(nr_covid19_images * test_split)], covid19_image_list_full[-round(nr_covid19_images * test_split):]]

# Combine lists of diffrent class images into single test/train list:
train_combined_lists_full = [normal_image_list_full[0], bacterial_pneumonia_image_list_full[0], viral_pneumonia_image_list_full[0], covid19_image_list_full[0]]
test_combined_lists_full = [normal_image_list_full[1], bacterial_pneumonia_image_list_full[1], viral_pneumonia_image_list_full[1], covid19_image_list_full[1]]

# Get datapoints and labels from images:
nr_train_samples = 200
x_data, y_data = get_dp_label_pair(train_combined_lists_full, nr_samples = nr_train_samples) 

nr_test_samples = 100
x_test, y_test = get_dp_label_pair(test_combined_lists_full, nr_samples = nr_test_samples)


Load pretrained models, train them on the training data set and evaluate them on the test data set: 

In [ ]:
# Load pretrained Keras CNNs without the fully connected layer weights (trained on imagenet-data):
VGG_model_wo_top = VGG(weights='imagenet', include_top=False)
ResNet_model_wo_top = ResNet(weights='imagenet', include_top=False)
DenseNet_model_wo_top = DenseNet(weights='imagenet', include_top=False)


In [ ]:
# Model structure parameters:
nr_classes = 4
input_shape = (256,256,3)
fc_layer_structure = [2048, 2048]

# Training parameters:
shuffle = True
batch_size = 1
epochs = 5

# Loss parameters:
weighted_loss = None


In [ ]:
vgg_model = redefine_model_wo_top(model = VGG_model_wo_top, input_shape = input_shape, nr_classes = nr_classes, fc_layer_structure = fc_layer_structure)
vgg_model, score = train_model(vgg_model, x_data, y_data, x_test = x_test, y_test = y_test, weighted_loss = weighted_loss, batch_size = batch_size, epochs = epochs, shuffle = shuffle)

w_f1_score, roc_auc, class_report, conf_matrix = evaluate_model(vgg_model, x_test, y_test, label_list = ["Normal", "Bacterial", "Viral", "Covid"])

print("\nWeighted F1-Score:", w_f1_score, "; ROC Curve and AUC:", roc_auc)
print("\nClassification Report:\n\n", class_report)
print("\nConfusion Matrix:\n\n", conf_matrix)


In [ ]:
resnet_model = redefine_model_wo_top(model = ResNet_model_wo_top, input_shape = input_shape, nr_classes = nr_classes, fc_layer_structure = fc_layer_structure)
resnet_model, score = train_model(resnet_model, x_data, y_data, x_test = x_test, y_test = y_test, weighted_loss = weighted_loss, batch_size = batch_size, epochs = epochs, shuffle = shuffle)

w_f1_score, roc_auc, class_report, conf_matrix = evaluate_model(resnet_model, x_test, y_test, label_list = ["Normal", "Bacterial", "Viral", "Covid"])

print("\nWeighted F1-Score:", w_f1_score, "; ROC Curve and AUC:", roc_auc)
print("\nClassification Report:\n\n", class_report)
print("\nConfusion Matrix:\n\n", conf_matrix)


In [ ]:
densenet_model = redefine_model_wo_top(model = DenseNet_model_wo_top, input_shape = input_shape, nr_classes = nr_classes, fc_layer_structure = fc_layer_structure)
densenet_model, score = train_model(densenet_model, x_data, y_data, x_test = x_test, y_test = y_test, weighted_loss = weighted_loss, batch_size = batch_size, epochs = epochs, shuffle = shuffle)

w_f1_score, roc_auc, class_report, conf_matrix = evaluate_model(densenet_model, x_test, y_test, label_list = ["Normal", "Bacterial", "Viral", "Covid"])

print("\nWeighted F1-Score:", w_f1_score, "; ROC Curve and AUC:", roc_auc)
print("\nClassification Report:\n\n", class_report)
print("\nConfusion Matrix:\n\n", conf_matrix)


Test predictions of each model for an random images from each class: 

In [ ]:
normal_test_image = random.choice(normal_image_list_full[0]) # Use normal train data set;

vgg_prediction = predict_xray_image(vgg_model, normal_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
resnet_prediction = predict_xray_image(resnet_model, normal_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
densenet_prediction = predict_xray_image(densenet_model, normal_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)

imgplot_xray = plt.imshow(normal_test_image)
plt.title("Label: Normal")
plt.show()

print("Predictions:\n")
print("VGG16:", vgg_prediction)
print("ResNet50V2:", resnet_prediction)
print("DenseNet121:", densenet_prediction)


In [ ]:
bacterial_pneumonia_test_image = random.choice(bacterial_pneumonia_image_list_full[0]) # Use bacterial train data set;

vgg_prediction = predict_xray_image(vgg_model, bacterial_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
resnet_prediction = predict_xray_image(resnet_model, bacterial_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
densenet_prediction = predict_xray_image(densenet_model, bacterial_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)

imgplot_xray = plt.imshow(bacterial_pneumonia_test_image)
plt.title("Label: Bacterial Pneumonia")
plt.show()

print("Predictions:\n")
print("VGG16:", vgg_prediction)
print("ResNet50V2:", resnet_prediction)
print("DenseNet121:", densenet_prediction)


In [ ]:
viral_pneumonia_test_image = random.choice(viral_pneumonia_image_list_full[0]) # Use viral train data set;

vgg_prediction = predict_xray_image(vgg_model, viral_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
resnet_prediction = predict_xray_image(resnet_model, viral_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
densenet_prediction = predict_xray_image(densenet_model, viral_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)

imgplot_xray = plt.imshow(viral_pneumonia_test_image)
plt.title("Label: Viral Pneumonia")
plt.show()

print("Predictions:\n")
print("VGG16:", vgg_prediction)
print("ResNet50V2:", resnet_prediction)
print("DenseNet121:", densenet_prediction)


In [ ]:
covid_pneumonia_test_image = random.choice(covid19_image_list_full[0]) # Use covid train data set;

vgg_prediction = predict_xray_image(vgg_model, covid_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
resnet_prediction = predict_xray_image(resnet_model, covid_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)
densenet_prediction = predict_xray_image(densenet_model, covid_pneumonia_test_image, label_list = ["Normal", "Bacterial", "Viral", "Covid"], display_results = False)

imgplot_xray = plt.imshow(covid_pneumonia_test_image)
plt.title("Label: Covid 19")
plt.show()

print("Predictions:\n")
print("VGG16:", vgg_prediction)
print("ResNet50V2:", resnet_prediction)
print("DenseNet121:", densenet_prediction)


Use an explainer (= LIME) on each trained classifier:

In [ ]:
# Define LIME-Explainer Instance:
explainer = lime_image.LimeImageExplainer()


In [ ]:
# Set specific indecies:
indecies = [150, 350, 550, 750]


In [ ]:
# Get Explanations for VGG16-Model:
fig, ax = explain_predictions(vgg_model, explainer, x_data, indecies = indecies, classes = ["Normal", "Bacterial", "Viral", "Covid"], num_features_lime = 10, num_samples_lime = 10000, figheight = 10, figwidth = 15)


In [ ]:
# Get Explanations for ResNet50V2-Model:
fig, ax = explain_predictions(resnet_model, explainer, x_data, indecies = indecies, classes = ["Normal", "Bacterial", "Viral", "Covid"], num_features_lime = 10, num_samples_lime = 10000, figheight = 10, figwidth = 15)


In [ ]:
# Get Explanations for DenseNet121-Model:
fig, ax = explain_predictions(densenet_model, explainer, x_data, indecies = indecies, classes = ["Normal", "Bacterial", "Viral", "Covid"], num_features_lime = 10, num_samples_lime = 10000, figheight = 10, figwidth = 15)


Store trained model in the model directory:

In [ ]:
# Save Keras-Models:
save_model(vgg_model, "vgg_model.h5")
save_model(resnet_model, "resnet_model.h5")
save_model(densenet_model, "densenet_model.h5")


In [ ]:
#------------------------------------------------------------------------------------------------------------------
# Get list of image from "normal chest xray" data directory:
normal_data_dir = os.path.join(data_dir_path, "Normal_old")
images = load_all_images_from_dir(normal_data_dir, (256, 256))[:1]
     
segment_lung_area(images)